In [3]:
import numpy as np
from gearbox_model import RxGearboxModel

In [4]:

header = [f' H{x}' for x in range(2)]
data = [f'D{x:02d}' for x in range(64)]
obuf = ['XXX' for _ in range(66)]


print('cnt / pause / head? / head_idx / data_idx / frame_word / obuf')

odata = []

for i in range(200):
    count = int(i % 33)
    pause = count == 32
    frame_word = int(count % 2 == 1)
    load_header = int(count % 2 == 0)
    header_idx = count if load_header else -1
    data_idx = count + 2 if load_header else count + 1


    
    # obuf[0:32] = obuf[32:64]
    # if not pause:
    #     if load_header:
    #         obuf[header_idx:header_idx+2] = header
    #     obuf[data_idx:data_idx+32] = data[frame_word*32 : 32 + frame_word*32]

    # odata.append(obuf[0:32])

    # bit select method
    for bit in range(len(obuf)):
        if bit < 32:
            obuf[bit] = obuf[bit+32]

        if not pause:
            if load_header:
                if bit >= header_idx and bit < header_idx + 2:
                    obuf[bit] = header[bit-header_idx]
                
            if bit >= data_idx and bit < data_idx + 32:
                if frame_word == 1:
                    obuf[bit] = data[(32 + bit) - data_idx]
                else:
                    obuf[bit] = data[bit - data_idx]


    odata.append(obuf[0:32])    

    print(f'{count:03d}\t{pause}\t{int(load_header)}\t{header_idx:02d}\t{data_idx:02d}\t{int(frame_word)}\t{obuf}')


cnt / pause / head? / head_idx / data_idx / frame_word / obuf
000	False	1	00	02	0	[' H0', ' H1', 'D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX']
001	False	0	-1	02	1	['D30', 'D31', 'D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62', 'D63', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 

In [5]:
rx_idata = odata.copy()

In [6]:
# rx

print('cnt / valid / head_idx / data_idx / frame_word / oheader / odata')

obuf = ['XXX' for _ in range(66)]
odata = []
oheader= []

for i in range(200):
    count = int(i % 33)
    valid = count != 0
    frame_word = int(count % 2 == 0)
    output_header = int(count % 2 == 1)
    idata = rx_idata[i]

    data_idxs = [00,32,64,30,62,28,60,26,58,24,56,22,54,20,52,18,50,16,48,14,46,12,44,10,42,8,40,6,38,4,36,2,34]

    # if count % 2 == 0:
    #     idata_idx = (66 - count) % 66
    #     obuf[idata_idx:66] = idata[0 : count]
    #     obuf[0:32-count] = idata[count:]

    # else:
    #     idata_idx = ((66 - count - 32) - 1) % 66
    #     obuf[idata_idx : idata_idx + 32] = idata

    # bit select method

    for bit in range(len(obuf)):

        idata_idx = data_idxs[count]

        if count % 2 == 0:
            if idata_idx != 0 and bit >= idata_idx and bit < 66:
                obuf[bit] = idata[bit-idata_idx]
            elif bit < 32 - count:
                obuf[bit] = idata[count + bit]
        else:
            if bit >= idata_idx and bit < idata_idx + 32:
                obuf[bit] = idata[bit - idata_idx]


    odata.append(obuf[2 : 34] if frame_word == 0 else obuf[34:66])
    oheader.append(obuf[0:2] if output_header else ['XXX','XXX'])

    print(f'{count:03d}\t{valid}\t{int(output_header)}\t{idata_idx:02d}\t{oheader[-1]}\t{odata[-1]}')

    

cnt / valid / head_idx / data_idx / frame_word / oheader / odata
000	False	0	00	['XXX', 'XXX']	['XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX']
001	True	1	32	[' H0', ' H1']	['D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31']
002	True	0	64	['XXX', 'XXX']	['D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62', 'D63']
003	True	1	30	[' H0', ' H1']	['D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D

In [10]:
rxg = RxGearboxModel('str')

for id in rx_idata:
    rxg.next(id)
    print(rxg.get_state())




000	False	0	00	[' H0', ' H1']	['XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX', 'XXX']
001	True	1	32	[' H0', ' H1']	['D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29', 'D30', 'D31']
002	True	0	64	[' H0', ' H1']	['D32', 'D33', 'D34', 'D35', 'D36', 'D37', 'D38', 'D39', 'D40', 'D41', 'D42', 'D43', 'D44', 'D45', 'D46', 'D47', 'D48', 'D49', 'D50', 'D51', 'D52', 'D53', 'D54', 'D55', 'D56', 'D57', 'D58', 'D59', 'D60', 'D61', 'D62', 'D63']
003	True	1	30	[' H0', ' H1']	['D00', 'D01', 'D02', 'D03', 'D04', 'D05', 'D06', 'D07', 'D08', 'D09', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'D16', 'D17', 'D18', 'D19', 'D20', 'D21', 'D22', 'D23', 'D24', 'D25', 'D26', 'D27', 'D28', 'D29

In [13]:
gen_idata = [np.random.randint(0,2,32) for _ in range(100)]

rxg = RxGearboxModel()

for id in gen_idata:
    rxg.next(id)
    print(rxg.get_state())

000	False	0	00	[1, 0]	[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
001	True	1	32	[1, 0]	[1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0]
002	True	0	64	[0, 1]	[1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1]
003	True	1	30	[0, 1]	[1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0]
004	True	0	62	[0, 1]	[0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0]
005	True	1	28	[0, 1]	[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1]
006	True	0	60	[1, 0]	[1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1]
007	True	1	26	[1, 0]	[0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0]
008	True	0	58	[0, 1]	[0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 